<a href="https://colab.research.google.com/github/ANSHUL-9268/PRODIGY_ML_03/blob/main/CAT_AND_DOG_CLASSIFIER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
import pickle

image_size = (224, 224)
train_dir = '/content/sample_data/dogs_and_cats'
model_filename = 'svm_model.pkl'

# Check for existing model
if os.path.exists(model_filename):
    # Load model
    with open(model_filename, 'rb') as f:
        clf = pickle.load(f)
    # Load pre-trained VGG16 model for feature extraction
    model = VGG16(weights='imagenet', include_top=False, input_shape=image_size + (3,))
else:
    # Load pre-trained VGG16 model
    model = VGG16(weights='imagenet', include_top=False, input_shape=image_size + (3,))

    # Extract features
    X, y = extract_features(train_dir)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train SVM
    clf = SVC(kernel='linear', C=1.0, random_state=42)
    clf.fit(X_train, y_train)

    # Save model
    with open(model_filename, 'wb') as f:
        pickle.dump(clf, f)


# Function to extract features
def extract_features(directory, sample_count=None):
    features = []
    labels = []
    for category in ['cats', 'dogs']:
        path = os.path.join(directory, category)
        for i, filename in enumerate(os.listdir(path)):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                img_path = os.path.join(path, filename)
                img = load_img(img_path, target_size=image_size)
                img_data = img_to_array(img)
                img_data = np.expand_dims(img_data, axis=0)
                img_data = preprocess_input(img_data)
                feature = model.predict(img_data)
                features.append(feature.flatten())
                labels.append(1 if category == 'dogs' else 0)
            if sample_count and (i + 1) >= sample_count:
                break
    return np.array(features), np.array(labels)

# Function to Classify an Image
def classify_image(image_path, model, clf):
    img = load_img(image_path, target_size=image_size)
    img_data = img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)
    features = model.predict(img_data)
    features = features.reshape(1, -1)  # Reshape features here
    prediction = clf.predict(features)[0]
    return "dog" if prediction == 1 else "cat"

# Example Usage: Replace with the path to your image
image_path = '/content/sample_data/dogs_and_cats/dogs/dog.1.jpg'
result = classify_image(image_path, model, clf)
print(f"The image is of a {result}")


1/1 [==============================] - 1s 1s/step
The image is of a dog


In [9]:
image_path = '/content/sample_data/dogs_and_cats/cats/cat.12.jpg'
result = classify_image(image_path, model, clf)
print(f"The image is of a {result}")

1/1 [==============================] - 0s 479ms/step
The image is of a cat


In [10]:
image_path = '/content/sample_data/dogs_and_cats/dogs/dog.16.jpg'
result = classify_image(image_path, model, clf)
print(f"The image is of a {result}")

1/1 [==============================] - 0s 465ms/step
The image is of a dog
